# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [25]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
session.execute("CREATE KEYSPACE IF NOT EXISTS events WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 3}")

#### Set Keyspace

In [8]:
session.set_keyspace('events')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Explanations for Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
For this we can choose sessionId as the partitionkey since it will effectively work in hashing just the specific items withihn that session to answer the query and use item_in_session as a clustering column since that narrows down the answer of all songs in that specific session history.

In [19]:
first_table ='CREATE TABLE events.song_by_session (session_id varint, item_in_session varint, artist_name text, song_name text, song_length float, PRIMARY KEY (session_id, item_in_session))'
session.execute(first_table)

In [20]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_by_session (session_id, item_in_session, artist_name, song_name, song_length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [21]:
first_query = 'SELECT artist_name, song_name, song_length FROM song_by_session WHERE session_id = 338 and item_in_session=4;'
rows = session.execute(first_query)
for result in rows:
    print(result)

Row(artist_name='Faithless', song_name='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


#### Explanations for Query 2:  Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
Again hashing with userid and sessionid which has specific requirements. Then we use iteminsession as the clustering column since we want a sort on the column and all the columns that follow.

In [22]:
second_table = 'CREATE TABLE events.song_by_user_session (user_id varint, session_id varint, item_in_session varint, artist_name text, song_name text, user_first_name text, user_last_name text, PRIMARY KEY ((user_id, session_id), item_in_session)) WITH CLUSTERING ORDER BY (item_in_session DESC)'
session.execute(second_table)

In [23]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_by_user_session (user_id, session_id, item_in_session, artist_name, song_name, user_first_name, user_last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [24]:
first_query = "SELECT artist_name, song_name, user_first_name, user_last_name FROM song_by_user_session WHERE user_id = 10 and session_id = 182;"
rows = session.execute(first_query)
for result in rows:
    print(result)

Row(artist_name='Lonnie Gordon', song_name='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Sebastien Tellier', song_name='Kilometer', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Three Drives', song_name='Greece 2000', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Down To The Bone', song_name="Keep On Keepin' On", user_first_name='Sylvie', user_last_name='Cruz')


#### Explanations for Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
This query doesnt really care about duplicates (whether a person listened to a song multiple times in different session or not). It just wants all the users (first name and last). We do want to capture uniqueness if theres two people named John Smith however. So we can use song_name as the partition key and user_id which promises a degree of PK uniqueness and squashes dimensions that dont matter down.

In [28]:
third_table = 'CREATE TABLE events.user_by_listen (song_name text, user_id varint, user_first_name text, user_last_name text, PRIMARY KEY (song_name, user_id))'         
session.execute(third_table)

In [29]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_by_listen (song_name, user_id, user_first_name, user_last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [30]:
first_query = "SELECT user_first_name, user_last_name FROM user_by_listen WHERE song_name = 'All Hands Against His Own';"
rows = session.execute(first_query)
for result in rows:
    print(result)

Row(user_first_name='Jacqueline', user_last_name='Lynch')
Row(user_first_name='Tegan', user_last_name='Levine')
Row(user_first_name='Sara', user_last_name='Johnson')


### Drop the tables before closing out the sessions

In [31]:
session.execute('DROP TABLE song_by_session')
session.execute('DROP TABLE song_by_user_session')
session.execute('DROP TABLE user_by_listen')

### Close the session and cluster connection¶

In [32]:
session.shutdown()
cluster.shutdown()